In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import random as python_random
import joblib

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import tensorflow as tf

from utils import *
import const

In [2]:
# reprodutividade
seed = 41
np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# obter dados do db
df = fetch_data_from_db(const.consulta_sql)

df.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24,58660.0,Outros,Ens.Médio,MuitoBom,58.0,0,Solteiro,VoyageRoamer,84623.00,350000.00,bom
1,Empresário,21,46557.0,Outros,Ens.Médio,MuitoBom,37.0,2,Víuvo,EcoPrestige,126855.00,500000.00,bom
2,Dentista,13,43939.0,Própria,Ens.Médio,Bom,23.0,0,Casado,DoubleDuty,127151.00,320000.00,ruim
3,Engenheiro,10,37262.0,Própria,Superior,Baixo,35.0,0,Divorciado,AgileXplorer,28767.00,250000.00,bom
4,Contador,6,52606.0,Própria,PósouMais,Justo,26.0,0,Casado,TrailConqueror,199564.00,400000.00,ruim


In [4]:
# conversões básicas
df['idade'] = df['idade'].astype(int)
df['valorsolicitado'] = df['valorsolicitado'].astype(float)
df['valortotalbem'] = df['valortotalbem'].astype(float)

In [5]:
# tratar nulos
tratar_nulos(df)

# tratar erros de digitacao
profissioes_validas = ['Advogado', 'Arquiteto', 'Cientista de Dados', 'Contador', 'Dentista', 'Engenheiro', 'Médico', 'Programador']
tratar_erros_digitacao(df, 'profissao', profissioes_validas)

# tratar outliers
df = tratar_outliers(df, 'tempoprofissao', 0, 70)
df = tratar_outliers(df, 'idade', 0, 110)

# criar features
df['propsolicitadototal'] = df['valorsolicitado']/df['valortotalbem']
df['propsolicitadototal'] = df['propsolicitadototal'].astype(float)

KeyError: 'coluna'

In [38]:
# divisão em treino teste e validacao
X = df.drop('classe', axis = 1)
y = df['classe']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [39]:
# normalização
X_test = save_scalers(X_test, ['tempoprofissao', 'renda', 'idade', 'dependentes', 'valorsolicitado', 'valortotalbem', 'propsolicitadototal'])
X_train = save_scalers(X_train, ['tempoprofissao', 'renda', 'idade', 'dependentes', 'valorsolicitado', 'valortotalbem', 'propsolicitadototal'])

In [40]:
# atribuir valores para variaveis categóricas
mapeamento = {'ruim': 0, 'bom':1}
y_train = np.array([mapeamento[item] for item in y_train])
y_test = np.array([mapeamento[item] for item in y_test])
X_train = save_encoders(X_train, ['profissao', 'tiporesidencia', 'escolaridade', 'score', 'estadocivil', 'produto'])
X_test = save_encoders(X_test, ['profissao', 'tiporesidencia', 'escolaridade', 'score', 'estadocivil', 'produto'])

In [41]:
# seleção de atributos
model = RandomForestClassifier()

In [47]:
# instancia do RFE
selector = RFE(model, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)


In [48]:
# transformar os dados
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
joblib.dump(selector, './objects/selector.joblib')

['./objects/selector.joblib']

In [50]:
# Modelo
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [51]:
# Otimizando 
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [53]:
# Compilar o modelo 
model.compile(optimizer= optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo
model.fit(
    X_train,
    y_train,
    validation_split=0.2, # 20% para validação
    epochs=500,
    batch_size=10,
    verbose=1
)

# Salvando o modelo
model.save('modelo_rnn.keras')

Epoch 1/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - accuracy: 0.9891 - loss: 0.0275 - val_accuracy: 0.7083 - val_loss: 5.7192
Epoch 2/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9724 - loss: 0.0365 - val_accuracy: 0.7500 - val_loss: 5.7294
Epoch 3/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0238 - val_accuracy: 0.7500 - val_loss: 5.7578
Epoch 4/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9614 - loss: 0.0382 - val_accuracy: 0.7500 - val_loss: 5.8223
Epoch 5/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9614 - loss: 0.0298 - val_accuracy: 0.7500 - val_loss: 5.8786
Epoch 6/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9891 - loss: 0.0352 - val_accuracy: 0.7500 - val_loss: 5.9243
Epoch 7/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9614 - loss: 0.0293 - val_accuracy: 0.7500 - val_loss: 6.0134
Epoch 8/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9614 - loss: 0.0314 - val_accuracy: 0.

In [54]:
# Previsões
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Avaliando o modelo
print("Avaliação do Modelo com os Dados de Teste:")
model.evaluate(X_test, y_test)

# Métricas de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
Avaliação do Modelo com os Dados de Teste:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.8333 - loss: 2.6753

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.88      0.64      0.74        11
           1       0.82      0.95      0.88        19

    accuracy                           0.83        30
   macro avg       0.85      0.79      0.81        30
weighted avg       0.84      0.83      0.83        30

